# install dependencies

In [ ]:
! pip install ultralytics

In [ ]:
!pip install omegaconf

In [ ]:
!pip install tensorflow-gpu

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
cd models/research

In [13]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git

In [ ]:
cd cocoapi/PythonAPI

In [ ]:
!make

In [17]:
cp -r pycocotools /content/models/research

In [ ]:
cd ..

In [ ]:
cd ..

In [21]:
cp object_detection/packages/tf2/setup.py .

In [ ]:
!python -m pip install .

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
cd ..

In [ ]:
cd ..

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!pip install ensemble-boxes

# Building the Pipeline

In [ ]:
CLASSES_MAP = {0:'wheel_mark',
               1:'equal_interval',
               2:'pavement',
               3:'pothole'}

SEVERITY_CLASSES_MAP = {0:'crack',
                        1:'insignificant',
                        2:'serious',
                        3:'tolerable'}

## Load Image

In [24]:
from PIL import Image
import numpy as np
import cv2

In [25]:
img = Image.open("path_to_test_image.jpg")
img_np = np.array(img)

In [ ]:
img_np = cv2.imread("path_to_test_image.jpg")
image_rgb = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)
image_expanded = np.expand_dims(image_rgb, axis=0)

# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(img_np)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]

## Load YOLO V8

In [3]:
from ultralytics import YOLO

In [4]:
# load yolov8 model
yolov8 = YOLO('v8.pt')

## Load YOLO V5

In [5]:
import torch

In [ ]:
cd yolov5

In [ ]:
yolov5 = torch.hub.load('ultralytics/yolov5', 'custom', path='../v5.pt')

In [14]:
cd ..

/content


## Load Faster RCNN Models

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import cv2
import argparse
from google.colab.patches import cv2_imshow

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# PROVIDE PATH TO MODEL DIRECTORY
PATH_TO_MODEL_DIR = '/content/FRCNN'  # Modify as needed


# PROVIDE PATH TO LABEL MAP
PATH_TO_LABELS = '/content/FRCNN/label_map.pbtxt'  # Modify as needed

# PROVIDE THE MINIMUM CONFIDENCE THRESHOLD
MIN_CONF_THRESH = float(0.40)

# LOAD THE MODELS

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL1 = PATH_TO_MODEL_DIR + "/FRCNN1"
PATH_TO_SAVED_MODEL2 = PATH_TO_MODEL_DIR + "/FRCNN2"

# LOAD SAVED MODEL AND BUILD DETECTION FUNCTION
detect_fn1 = tf.saved_model.load(PATH_TO_SAVED_MODEL1)
detect_fn2 = tf.saved_model.load(PATH_TO_SAVED_MODEL2)


## Define Prediction Functions

In [37]:
def yolov8_prediction():
  results = yolov8(img)
  results = results[0]
  results = results.cpu()
  boxes = results.boxes.xyxyn.tolist()  # box with xyxy format but normalized, (N, 4)
  scores = results.boxes.conf.tolist()   # confidence score, (N, 1)
  classes = results.boxes.cls.tolist()    # cls, (N, 1)
  boxes = [[box[1], box[0], box[3], box[2]] for box in boxes]  # convert to yxyx format
  return boxes, scores, classes

In [ ]:
import pandas as pd

def yolov5_prediction():
  results = yolov5(img)
  results = results.pandas().xyxy[0]

  h, w = img_np.shape[0], img_np.shape[1]

  results['ymin'] = results['ymin'] / h
  results['ymax'] = results['ymax'] / h
  results['xmin'] = results['xmin'] / w
  results['xmax'] = results['xmax'] / w

  boxes = results[['ymin', 'xmin', 'ymax', 'xmax']].values.tolist()
  scores = results.confidence.values.tolist()
  classes = results['class'].values.tolist()

  detectionsDect = {'boxes': boxes, 'scores': scores, 'classes': classes}

  return boxes, scores, classes

In [ ]:
def frcnn_1_prediction():
  # input_tensor = np.expand_dims(image_np, 0)
  FRCNN_1_detections = detect_fn1(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.

  num_detections1 = int(FRCNN_1_detections.pop('num_detections'))
  FRCNN_1_detections = {key: value[0, :num_detections1].numpy()
                for key, value in FRCNN_1_detections.items()}
  FRCNN_1_detections['num_detections'] = num_detections1

  # detection_classes should be ints.
  FRCNN_1_detections['detection_classes'] = FRCNN_1_detections['detection_classes'].astype(np.int64)


  boxes = np.squeeze(FRCNN_1_detections['detection_boxes']).tolist() if len(FRCNN_1_detections['detection_boxes']) != 1 else [np.squeeze(FRCNN_1_detections['detection_boxes']).tolist()]
  scores = np.squeeze(FRCNN_1_detections['detection_scores']).tolist() if len(FRCNN_1_detections['detection_scores']) != 1 else [np.squeeze(FRCNN_1_detections['detection_scores']).tolist()]
  classes = np.squeeze(FRCNN_1_detections['detection_classes']).tolist() if len(FRCNN_1_detections['detection_classes']) != 1 else [np.squeeze(FRCNN_1_detections['detection_classes']).tolist()]

  return boxes, scores, classes

In [ ]:
def frcnn_2_prediction():
  # input_tensor = np.expand_dims(image_np, 0)
  FRCNN_2_detections = detect_fn2(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.

  num_detections2 = int(FRCNN_2_detections.pop('num_detections'))
  FRCNN_2_detections = {key: value[0, :num_detections2].numpy()
                for key, value in FRCNN_2_detections.items()}
  FRCNN_2_detections['num_detections'] = num_detections2

  # detection_classes should be ints.
  FRCNN_2_detections['detection_classes'] = FRCNN_2_detections['detection_classes'].astype(np.int64)


  boxes = np.squeeze(FRCNN_2_detections['detection_boxes']).tolist() if len(FRCNN_2_detections['detection_boxes']) != 1 else [np.squeeze(FRCNN_2_detections['detection_boxes']).tolist()]
  scores = np.squeeze(FRCNN_2_detections['detection_scores']).tolist() if len(FRCNN_2_detections['detection_scores']) != 1 else [np.squeeze(FRCNN_2_detections['detection_scores']).tolist()]
  classes = np.squeeze(FRCNN_2_detections['detection_classes']).tolist() if len(FRCNN_2_detections['detection_classes']) != 1 else [np.squeeze(FRCNN_2_detections['detection_classes']).tolist()]

  return boxes, scores, classes


In [18]:
from ensemble_boxes import weighted_boxes_fusion

def ensemble(boxes, scores, classes, weights):
    boxes = [[[box[1], box[0], box[3], box[2]] for box in model] for model in boxes] # convert to xyxy format

    iou_thr = 0.5
    skip_box_thr = 0.0001
    boxes, scores, classes = weighted_boxes_fusion(boxes, scores, classes, weights=weights, iou_thr=iou_thr,
                                                   skip_box_thr=skip_box_thr)

    boxes = np.squeeze(boxes).tolist() if len(boxes) != 1 else [np.squeeze(boxes).tolist()]
    scores = np.squeeze(scores).tolist() if len(scores) != 1 else [np.squeeze(scores).tolist()]
    classes = np.squeeze(classes).tolist() if len(classes) != 1 else [np.squeeze(classes).tolist()]

    return boxes, scores, classes

In [20]:
def potholes_localization_pipeline():
  v8_boxes, v8_scores, severities = yolov8_prediction()
  v5_boxes, v5_scores, classes = yolov5_prediction()
  f1_boxes, f1_scores, f1_classes = frcnn_1_prediction()
  f2_boxes, f2_scores, f2_classes = frcnn_2_prediction()

  boxes = [v8_boxes, v5_boxes, f1_boxes, f2_boxes]
  scores = [v8_scores, v5_scores, f1_scores, f2_scores]
  all_classes = [severities, classes, f1_classes, f2_classes]

  all_classes = [[1 for cls in clss] for clss in all_classes]

  boxes, scores, _ = ensemble(boxes, scores, all_classes, [2, 1, 1, 1])


  return boxes, scores, classes, severities

## Dimension Estimation Module

In [ ]:
CALIBRATED_FOCAL_LENGTH = 10  # Calculate the focal length as described in the provided write-up (The provided number is just for the demo)
CAMERA_ELEVATION_FROM_GROUND = 90  # We assumed that the camera is at 90cm elevation.

In [ ]:
# We will calculate the pothole width and length and areal with the following equation.
# But first we have to find pixel width and length for the pothole

# pothole_width = pixel_width * CAMERA_ELEVATION_FROM_GROUND / CALIBRATED_FOCAL_LENGTH

In [39]:
def get_true_dimension(pixel_dim):
  return pixel_dim * CAMERA_ELEVATION_FROM_GROUND / CALIBRATED_FOCAL_LENGTH

In [19]:
def get_pothole_dimensions(boxes):
  width_list = []
  length_list = []
  area_list = []

  for box in boxes:
    width_list.append(box[2] - box[0])
    length_list.append(box[1] - box[3])

  width_list = list(map(get_true_dimension, width_list))
  length_list = list(map(get_true_dimension, length_list))

  area_list = [w * h for w, h in zip(width_list, length_list)]


  return width_list, length_list, area_list

## Severity Classification Pipeline

In [ ]:
# Weights should be set be domain experts and are provided here just for the demo

CLASS_WEIGHT_MAP = {'wheel_mark': 0.2,
                    'equal_interval': 0.2,
                    'pavement': 0.5,
                    'pothole': 1}

SEVERITY_WEIGHT_MAP = {'crack': 0.7,
                       'insignificant':0.8,
                       'tolerable':0.9,
                       'serious':1}

In [ ]:
def harmonic_ranking(area, class_weight, severity_weight):
  return 3 /((1/area) + (1/class_weight) + (1/severity_weight))

In [ ]:
def severity_scores_pipeline():
  boxes, _, classes, severities = potholes_localization_pipeline()
  _, _, area_list = get_pothole_dimensions(boxes)

  classes_weights = list(map(lambda x: CLASS_WEIGHT_MAP[CLASSES_MAP[x]], classes))
  severities_weights = list(map(lambda x: SEVERITY_WEIGHT_MAP[SEVERITY_CLASSES_MAP[x]], severities))

  potholes_properties = zip(area_list, classes_weights, severities_weights)
  
  severity_scores = list(map(harmonic_ranking, potholes_properties))

  return list(zip(boxes, severity_scores))

In [22]:
# call severity_scores_pipeline after loading the models and the image to get the severity score (out of 1) for each pothole in the image

In [ ]:
severity_scores = severity_scores_pipeline()  # This returns each box coordinate with its severity score